<a href="https://colab.research.google.com/github/prushton2/VexScoringProgram/blob/master/RoboticsScoreProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Resources:

https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=3jKM6GfzlgpS
https://colab.research.google.com/notebooks/widgets.ipynb

# Class definitions

## Json manager class

In [0]:
import matplotlib.pyplot as plt
import csv
import json
from termcolor import colored, cprint 
import time 
import os
import sys
import google.colab as colab


null = None #I can't be bothered to remember null is None in python, and booleans are capital
true = True
false = False


class JsonManager:
  def __init__(self, path, fType): # fType should be 'score', 'comp', or 'team' depending on which json file is being opened Currently unused
    self.path = path
    self.fType = fType
  def load(self):
    #opens json file and returns data
    with open(self.path, mode="r") as theFile:
      return json.load(theFile) 
      
  def save(self, data):
    #opens json file and saves data
    with open(self.path, mode="w") as theFile:
      json.dump(data, theFile)

  def addData(self, key, value, function): #Adds data to a json key, but abstracted. 
    jsonFile = self.load()#JsonManager.open()

    if(function == 'set'): #Higher level way to set json. Used to be useful, not as useful now
      exec("jsonFile"+str(key)+" = "+str(value))
    elif(function == 'append'):
      exec("jsonFile"+str(key)+".append("+str(value)+")")
    else:
      raise ValueError("The function parameter must be 'set' or 'append', not", function)
    self.save(jsonFile)

  

## Competition Manager

In [0]:
class Comp: #This is used as a datatype to hold the data for a competition
  def __init__(self, id, name, date, path):
    self.id = id;
    self.name = name;
    self.date = date;
    self.path = path;

  

class AllComps: #This is to organize the comps
  def __init__(self, compList):
    self.compList = compList;
  def getCompFromId(self, id):
    for i in self.compList:
      if(i.id == id):
        return i

## Compiler

The compiler class will take the CSV files and will format and store them in the json files.

In [0]:
class Compiler:
  def __init__(self, comps):
    self.comps = comps;

  def setCompIDs(self, comps):
    self.comps = comps
  
  def compileComps(self): # Will compile the csv data into the comp json file.
    for thisComp in self.comps:

      compFile.addData('['+str(thisComp.id)+']', {"Matches":[],"Skills":[]}, 'set') #Initialize the comp in the json file

      with open(thisComp.path, mode='r') as thisFile: #Open the respective csv file
        reader = csv.DictReader(thisFile)
        line_count=0

        for row in reader: #Iterate through each row, as each row represents a match
          print("Adding match", row["Match"], "of comp", thisComp.name)

          if(int(row["Red Score"]) > int(row["Blue Score"])): #I find the winning team and store it in the json for easy use later
            winner = 'Red'
          elif(int(row["Blue Score"]) > int(row["Red Score"])):
            winner = 'Blue'
          else:
            winner = null

          thisMatch = [ #This is the jagged list that stores the entire competition
            row["Match"],
            thisComp.id, 
            [int(row["Red Score"]), int(row["Blue Score"])],
            [row["Red Team (1)"], row["Red Team (2)"]], 
            [row["Blue Team (1)"], row["Blue Team (2)"]],
            winner
          ]

          compFile.addData('["'+str(thisComp.id)+'"]["Matches"]', thisMatch, 'append') #Add the comp list to the json file
  
  def compileTeams(self): # This compiles every match every team has played into the teams json. Format is in the structure block
    json = compFile.load() #I iterate through the compFile json because it is easier for me to write
    for i in json: #For each comp 
      for j in json[i]["Matches"]: #For each match
        for k in j[3]:# For each red alliance team
          try: #If we can't add the data to the team, the team probably doesnt exist. The try statement will create the team if the team doesnt exist already
            teamFile.addData("['"+k+"']['Matches']", j, 'append')
            print("Added", j, "to team", k)
          except:
            teamFile.addData("['"+k+"']", {"Matches":[j],"Skills":[]}, 'set')
            print("Created team", k, "with match", j)
        for k in j[4]: #For each blue alliance team
          try:
            teamFile.addData("['"+k+"']['Matches']", j, 'append')
            print("Added", j, "to team", k)
          except:
            teamFile.addData("['"+k+"']", {"Matches":[j],"Skills":[]}, 'set')
            print("Created team", k, "with match", j)

  def compileScores(self): #This calculates and saves the scores into a json file. The score class is responsible for ranking the teams and re-saving the json
    scores = scoreFile.load();
    teams = teamFile.load();
    for team in teams: #Iterate through each team
      scoreFile.addData("['"+team+"']", [0], 'set') #Initialize each team with a score of 0
      print("Created a record for team", team, "with 0 points")
      for match in teams[team]["Matches"]:
        scores = scoreFile.load();

        if(match[5] == 'Red'): #Create the number to add/subtract to the players score, saved as delta
          delta =  (int(match[2][0]) / (int(match[2][0]) + int(match[2][1])))*100 #The logic for this math is in the structure blockk
        if(match[5] == 'Blue'):
          delta =  (int(match[2][1]) / (int(match[2][0]) + int(match[2][1])))*100
        if(match[5] == null):
          delta = 0
        
        if(not(MatchHandler.hasWon(team, match))): #Invert the value if the team lost, making them lose points if they lost
          delta *= -1;


        z = (scores[team][-1] + delta) #I used the z variable because it kept the code readable
        scoreFile.addData('["'+team+'"]', z, 'append')
        print("Added", delta, "points to the record for team", team)

          
        


## Match Handler

In [0]:
class MatchHandler: #Basic static class, used to make my life easier in coding the compilers
  @staticmethod
      
  def getColor(team, match):
    if(team == match[3][0] or team == match[3][1]):
      return "Red"
    if(team == match[4][0] or team == match[4][1]):
      return "Blue"

  def hasWon(team, match):
    if(MatchHandler.getColor(team, match) == match[5]):
      return True
    else:
      return False # set cube stack

## Team Class

In [0]:
class Team:
  def __init__(self, teamName):
    self.team = teamName;

  def hasCategory(self, category, categories):
    return category in categories or "all" == categories

  def showStats(self, categories):
    if(self.hasCategory("matches", categories)):
      teams = teamFile.load();
      for i in teams[self.team]["Matches"]:
        print(allComps.getCompFromId(i[1]).name, "on", allComps.getCompFromId(i[1]).date, "   |   ", end="")
        
        if(MatchHandler.hasWon(self.team, i)):
          color='green'
        elif(i[5] == null):
          if(i[2][0] == 0 and i[2][1] == 0):
            color = 'yellow'
          else:
            color='grey'
        else:
          color='red'
        
        print(colored(i[0], color, attrs=['reverse', 'blink']), end='')
        print("  |  ", end='')

        if(i[5] == null):
          if(i[2][0] == 0 and i[2][1] == 0):
            print(colored(i[3][0]+", "+i[3][1]+" : "+str(i[2][0])+" vs "+str(i[2][1])+" : "+i[4][0]+", "+i[4][1], 'yellow'), end='')            
          else:
            print(colored(i[3][0]+", "+i[3][1]+" : "+str(i[2][0])+" vs "+str(i[2][1])+" : "+i[4][0]+", "+i[4][1], 'grey'), end='')

        if(i[5] == 'Red'):
          print(colored(i[3][0]+", "+i[3][1]+" : "+str(i[2][0])+" vs ", 'red'), end='')
          print(str(i[2][1])+" : "+i[4][0]+", "+i[4][1], end='')

        if(i[5] == 'Blue'):
          print(i[3][0]+", "+i[3][1]+" : "+str(i[2][0]), end='')
          print(colored(" vs "+str(i[2][1])+" : "+i[4][0]+", "+i[4][1], 'blue'), end='')

        print("")

    if(self.hasCategory("scores", categories)):
      mplCreator.newPlot(false, s1.getScores(self.team), 'Scores')

    if(self.hasCategory("rank", categories)):
      print(self.team, "is placed", str(s1.getRank(self.team))+"/"+str(s1.getTeamCount()), end='')   

    if(self.hasCategory("score", categories)):
      if(self.hasCategory("rank", categories)):
        print(" and has", s1.getScore(self.team), "points")
      else:
        print(self.team, "has", s1.getScore(self.team), 'points')
    print('')
    
        
        
          

## Matplotlib Creators

In [0]:
class mplCreator: #Makes a dark theme plot of two given arrays. The plot also draws lines between the points with different colors
  @staticmethod
  def getArrayOfLength(length):
    array = []
    for i in range(length):
      array.append(i)
    return array

  def newPlot(xArray, yArray, title):
    fig, ax = plt.subplots()
    #Create a plot instance
    
    #make it dark
    # https://matplotlib.org/3.1.1/tutorials/introductory/customizing.html
    plt.rcParams.update({
        "lines.color": "white",
        "patch.edgecolor": "white",
        "text.color": "black",
        "axes.facecolor": "black",
        "axes.edgecolor": "lightgray",
        "axes.labelcolor": "white",
        "xtick.color": "white",
        "ytick.color": "white",
        "grid.color": "lightgray",
        "figure.facecolor": "black",
        "figure.edgecolor": "black",
        "savefig.facecolor": "black",
        "savefig.edgecolor": "black"})
    #Place points on the graph
    if(xArray == False and yArray == False):
      print("We need a list, not 2 booleans")
    if(xArray == False):
      xArray = mplCreator.getArrayOfLength(len(yArray))
    if(yArray == False):
      yArray = mplCreator.getArrayOfLength(len(xArray))

    for i in range(len(yArray)):#Color in the lines fron point to point based on gain or loss
      try:
        if(yArray[i] > yArray[i+1]):
          ax.plot([xArray[i], xArray[i+1]], [yArray[i], yArray[i+1]], color='r')
        elif(yArray[i] < yArray[i+1]):
          ax.plot([xArray[i], xArray[i+1]], [yArray[i], yArray[i+1]], color='g')
        else:
          ax.plot([xArray[i], xArray[i+1]], [yArray[i], yArray[i+1]], color='b')
      except:
        print("")
    ax.scatter(xArray, yArray)
    #Set the title
    ax.set_title(title)
    #Show the plot
    fig.show()

## Score Class

In [0]:
class Score:
  def __init__(self):
    self.Ranks = []
  

  def rankTeams(self):
    IMM_scores = scoreFile.load()
    self.Ranks = []
    self.teams = []
    self.newScores = {}
    for i in IMM_scores:
      self.teams.append(i)
    print(self.teams)

    for team in IMM_scores:
      teamScore = IMM_scores[team][-1]
      rankFound = false
      print(team)

      for i in range(len(self.Ranks)):
        print(self.Ranks[i], ":", IMM_scores[self.Ranks[i]][-1], "||", end='')


      if(self.Ranks == []):
        self.Ranks = [team]
        print("--- Initialized ranks with team", team)
      else:
        i=-1
        while(not(rankFound)):
          i += 1
          # print("--- Index:", i)
          try:
            comparisonScore = IMM_scores[self.Ranks[i]][-1]
            # print("--- Comparison: Team:", self.Ranks[i], "Score:", comparisonScore)
            # print("--- This: Team:", team, "Score:", teamScore)

            if(teamScore > comparisonScore):
              # print("---", teamScore, ">", comparisonScore)
              print("--- found a spot for", team, "at place", i)
              rankFound = true
              self.Ranks.insert(i, team)
            else:
              pass
              # print("---", teamScore, "<", comparisonScore)
          except:
            rankFound = true
            self.Ranks.append(team)
            print("--- Hit list limit at", i, "appending", team, "to ranks list")
            i=-1
      pass
      self.newScores = {}
      for i in self.Ranks:
        self.newScores[i] = IMM_scores[i]
        # print(i, newScores[i])
      scoreFile.save(self.newScores)
    
  def getRank(self, givenTeam):
    scores = scoreFile.load()
    i = 0
    for team in scores:
      i += 1
      if(givenTeam == team):
        return i

  def getScore(self, givenTeam):
    scores = scoreFile.load()
    for team in scores:
      if(givenTeam == team):
        return scores[team][-1]

  def getScores(self, givenTeam):
    scores = scoreFile.load()
    for team in scores:
      if(givenTeam == team):
        return scores[team]

  def getTeamCount(self):
    i=0
    scores = scoreFile.load()
    for j in scores:
      i += 1
    return i

  def showRanks(self):
    scores = scoreFile.load()
    i = 0
    for team in scores:
      i += 1
      print(str(i)+":", team, "with a score of", scores[team][-1])

## Printer

In [0]:
class Printer:
  def __init__(self):
    self.List = []

  def reset(self):
    self.List = []

  def Print(self):
    Printer.clear(self)
    for i in self.List:
      for j in i:
        print(j, end='')
      print('')

  def clear(self): 
    colab.output.clear()

# Instantiations

In [0]:
localpath = "/content/drive/My Drive/RoboticsScoreProgram" #If you are using this on your local machine, change the path to fit your case
compList = [
            Comp(0.0,  "CTEEA Hall High School",              "10/19/19", localpath+"/Competitions/VRC_10_19_19_Hartford.csv"),
            Comp(1.0,  "QCC Fall Classic",                    "11/02/19", localpath+"/Competitions/VRC_11_02_19_Madison.csv"),
            Comp(2.0,  "Daniel Hand High School",             "11/02/19", localpath+"/Competitions/VRC_11_02_19_QCC.csv"),
            Comp(3.0,  "Framingham",                          "11/09/19", localpath+"/Competitions/VRC_11_09_19_Framingham.csv"),
            Comp(4.0,  "Danbury High",                        "11/16/19", localpath+"/Competitions/VRC_11_16_19_Monroe.csv"),
            Comp(5.0,  "North Andover",                       "11/23/19", localpath+"/Competitions/VRC_11_23_19_NorthAndover.csv"),
            Comp(6.0,  "Hopkinton",                           "12/07/19", localpath+"/Competitions/VRC_12_07_19_Hopkinton.csv"),
            Comp(7.0,  "CTEEA Masuk",                         "12/07/19", localpath+"/Competitions/VRC_12_07_19_Masuk.csv"),
            Comp(8.0,  "QCC Winter Classic",                  "12/21/19", localpath+"/Competitions/VRC_12_21_19_QCC.csv"),
            Comp(9.0,  "CTEEA Bolton CT",                     "01/04/20", localpath+"/Competitions/VRC_01_04_20_CTEEABolton.csv"),
            Comp(10.0, "Taking the New Year's Tower at QCC",  "01/04/20", localpath+"/Competitions/VRC_01_04_20_QCC.csv"),
            Comp(11.0, "CTEEA Middleton",                     "01/11/20", localpath+"/Competitions/VRC_01_11_20_CTEEAMiddleton.csv"),
            Comp(12.0, "The Wave at WPI",                     "01/16/20", localpath+"/Competitions/VRC_01_16_20_Wave.csv"),
            Comp(13.0, "Hopkinton",                           "01/19/20", localpath+"/Competitions/VRC_01_19_20_Hopkinton.csv"),
            Comp(14.0, "North Andover",                       "01/25/20", localpath+"/Competitions/VRC_01_25_20_NorthAndover.csv")

]

allComps = AllComps(compList)
compiler = Compiler(compList)

s1 = Score()
printer = Printer()

compFile = JsonManager(localpath+"/Json/Comps.json", 'comp')
teamFile = JsonManager(localpath+"/Json/Teams.json", 'team')
scoreFile = JsonManager(localpath+"/Json/Scores.json", 'score')

# Compiling
Compiles the CSV files into the JSON files
Only run if the CSV files were updated, or the compiler code was changed

In [0]:
if(false): #This compiles the CSV files to the JSON files. Do not run unless you have made changes to the CSV files or the compiler code
  startTime = time.time()
  compFile.save({})
  teamFile.save({})
  scoreFile.save({})

  compiler.compileComps()
  compiler.compileTeams()
  compiler.compileScores()

  s1.rankTeams()

  print("Operation took", time.time()-startTime,"seconds")


# Show teams with the given number

In [36]:
teams = teamFile.load()
Team_Number = "16" #@param {type:"string"}
printer.reset()
for i in teams:
  if Team_Number in i:
    printer.List.append([i])
printer.Print()



1695C
1697B
1697A
1695B
1695A
1695E
1695D
169Y
4716A
4716D
1618A
162A
16738B
1615A
4716C
4716B
169Z
169E
2616Z
169A
169X
16738A


# Show a team's scores

In [0]:
team = "977Z" #@param {type:"string"}
Matches = True #@param {type:"boolean"}
Score = True #@param {type:"boolean"}
ScorePlot = True #@param {type:"boolean"}
Rank = True #@param {type:"boolean"}

List = []

if(Matches):
  List.append("matches")
if(Score):
  List.append("score")
if(ScorePlot):
  List.append("scores")
if(Rank):
  List.append("rank")

aTeam = Team(team)

aTeam.showStats(List) #Instead of all, can be a list containing: 'matches', 'scores', 'score', 'rank'

# Show scores of teams in a list

In [0]:
teamList = [ "785A", "785B", "817B", "817F", "977Z", "2442A", "2442E", "2442F", "2602A", "2602B", "2602C", "2602D", "2602E", "2602F", "2602G", "2602H", "2602K", "2602M", "2606A", "2606B", "2606C", "3379B", "4344Y", "4344Z", "4898A", "4898B", "4898C", "5313A", "5313B", "5313C", "8568A", "8568B", "8568C", "8568D", "8568E", "8568F", "8568G", "8568H", "8568J", "8568K", "8568X", "8884A", "8884B", "9421R", "9421V", "9421X", "16738A", "16738B", "16738C", "25600H", "25600X", "29718A", "63857A", "63857B", "63857D", "63857E", "76645A", "99928A", "99928B", "99928C", "99928D", "99928E" ] #@param {type:"raw"}
compName = "North Andover" #@param {type:"string"}

print(f"Everyone going to {compName} on 1/25/20")
print('')
print("Predicted rank, Current rank, Number, Score")
print('')

scores = scoreFile.load()
i = 0
for team in scores:
  if team in teamList:
    i += 1
    print(i, "|", s1.getRank(team), "|", team, "|", s1.getScore(team))

# Show SNE team rankings

In [0]:
s1.showRanks();

# Structure
* Json Files
  * Competitions:
    * Holds every played match sorted by competition then time
  * Scores
    * Holds every team's score, and their score over time stored in a list
  * Teams
    * Stores every team, and every match they have played.

# Backlog

### Complete
* Create Json manager class
* Create competition class
* Create competition ID system
* Read and compile csvs into jsons
  * Have a json for comps and a json for teams
* Create scoring system
* Print a teams stats nicely
* Show scores over time

### In progress
* Display competitions nicely

### TODO
.
# Sprint Task List
* Implement skills

#### Complete
.
#### In progress
* Create comp class

#### TODO
* Use similar print format from the teams match printing
* Allow multiple comps to be shown at once
* Allow highlighting of certain teams
  * Also implement into the teams class

# Json format:

## Comps:
    {
      "CompName":{
        "Matches":[
          [
            "matchName",
            compId,
            [redScore, blueScore],
            [red1, red2],
            [blue1, blue2],
            "winner"
          ]
        ],
        "Skills": [
          [
            "TeamName",
            [autonScore, driverScore]
          ]
        ]
      }
    }
## Scores:

    {
      "teamNumber": [
        score1,
        score2,
        score3...
      ]
    }

## Teams:

    {
      "teamNumber": {
        "Matches: [
          "referenceToCompsJsonMatchIndex",
        ],
        "Skills": [

        ]
      }
    }


